In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
from matplotlib.pyplot import figure 
from sklearn import cluster
from sklearn import metrics
from sklearn.preprocessing import scale
pd.set_option('display.precision',3)
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 300
pd.options.display.float_format = '{:20,.2f}'.format
from keras.models  import Sequential, K
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam, SGD, RMSprop
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score, roc_curve, accuracy_score
from sklearn.ensemble import RandomForestClassifier
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import seaborn as sns
%matplotlib inline

In [0]:
#### Replaceable code ##### 
#df1=pd.read_csv('cf.csv')
#df2=pd.read_csv('orig.csv')

In [0]:
######## The code below is primeraly used for executing in google colabs - pls replace the google colabs code with the commented code above if required###########

##### Start of google colabs code ############
from google.colab import drive
drive.mount('/content/drive')

In [0]:
## Upload cf.csv file ##
from google.colab import files
uploaded = files.upload()

In [0]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
uploaded

In [0]:
import pandas as pd
import io
df1 = pd.read_csv(io.StringIO(uploaded['cf.csv'].decode('utf-8')))
print(df1)

In [0]:
### Upload orig.csv ####
uploaded2 = files.upload()

In [0]:
for fn in uploaded2.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded2[fn])))

In [0]:
uploaded2

In [0]:
df2 = pd.read_csv(io.StringIO(uploaded2['orig.csv'].decode('utf-8')))
print(df2)

In [0]:
## End of google colabs code ####

In [0]:
#Changing the order of the 3 specified columns

df1= df1[[c for c in df1 if c not in ['time', 'viewdoc', 'viewtopic']] 
       + ['time', 'viewdoc', 'viewtopic']]


df1.head()

In [0]:
print(df1.shape)
print(df2.shape)
df = pd.concat([df1,df2])
df.head(10)

In [0]:
df.columns

In [0]:
df.shape

In [0]:
#Create rate column
df.loc[df['TREL'] == df['urel'], 'rate'] = 1
df.loc[df['TREL'] != df['urel'], 'rate'] = 0

In [0]:
#Create New size column
df.loc[df['SIZE'] == "lrg", 'Nsize'] = 2
df.loc[df['SIZE'] == "med", 'Nsize'] = 1
df.loc[df['SIZE'] == "sml", 'Nsize'] = 0
df

In [0]:
df.columns

In [0]:
################ Dropping the columns #################################

In [0]:
df= df.drop(columns=['USER', 'GENDER',	'COUNTRY',	'ENGLISH',	'EDUCATION',	'SIZE',	'TREL',	'TOPIC',	'DOC', 'viewdoc',	'viewtopic', 'AGE'	])
df

In [0]:
df= df.reset_index(drop=True)
df.tail()

In [0]:
df_null=df.isnull().T.sum() #checking the rows with max NAN values
df_null=df_null.reset_index(name='val')
df_null

In [0]:
df_null.index[df_null['val'] > 0 ].tolist()

In [0]:
########### Dropping the null columns #################
df= df.drop([324, 325, 326, 327, 328, 329, 330, 331, 332, 845])

In [0]:
df= df.reset_index(drop=True)
df.tail()

In [0]:
df.head()

In [0]:
############### Splitting the data into x and y ######################

In [0]:
df_x=df.iloc[:, [0,1,2,3,4,5,6,7,9]].astype('float').values
df_x

In [0]:
df_y=df.iloc[:, [8]].values
df_y

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.25, random_state=11111)

In [0]:
############### Neural Network Architecture- 1  ###################

In [0]:
# set random seed 
seed = 7
seed = numpy.random.seed(seed)

In [0]:
# Perform Label encoding on y data
encoder = LabelEncoder()
encoder.fit(df_y)
encoded_Y = encoder.transform(df_y)

In [0]:
# baseline model
def arch1():
# create model
  model = Sequential()
  model.add(Dense(9, input_dim=9, kernel_initializer='normal', activation='tanh'))
  model.add(Dropout(0.5))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
  return model

In [0]:
# Running Architecture- 1
estimator = KerasClassifier(build_fn=arch1, epochs=100, batch_size=10, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, df_x, df_y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [0]:
# arch1 Model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=arch1, epochs=50, batch_size=10, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, df_x, df_y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [0]:
###############  Architecture- 2  ###################

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.25, random_state=11111)

In [0]:
np.mean(df_y), np.mean(1-df_y)

In [0]:
### Random Forest Classifier ###
rf_model = RandomForestClassifier(n_estimators=200)
rf_model.fit(X_train, y_train)

In [0]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(rf_model, df_x, df_y, cv=kfold)
print("Random Forest: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [0]:
### Testing Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier
rf_model = GradientBoostingClassifier(n_estimators=200)
rf_model.fit(X_train, y_train)

In [0]:
kfold = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed)
results = cross_val_score(rf_model, df_x, df_y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [0]:
###############  Architecture- 3  ###################

In [0]:
# larger model
def arch3():
	# create model
	model = Sequential()
	model.add(Dense(9, input_dim=9, kernel_initializer='normal', activation='tanh'))
	model.add(Dense(5, kernel_initializer='normal', activation='tanh'))
	model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='RMSprop', metrics=['accuracy'])
	return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=arch3, epochs=100, batch_size=10, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, df_x, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [0]:
# larger model
def arch3():
	# create model
  model = Sequential()
  model.add(Dense(9, input_dim=9, kernel_initializer='normal', activation='tanh'))
  model.add(Dropout(0.5))
  model.add(Dense(5, kernel_initializer='normal', activation='tanh'))
  model.add(Dropout(0.5))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=arch3, epochs=100, batch_size=10, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, df_x, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [0]:
###############  Neural Network Architecture- 4  ################### 

In [0]:
def arch4():
	# create model
  model = Sequential()
  model.add(Dense(9, input_dim=9, kernel_initializer='normal', activation='tanh'))
  model.add(Dropout(0.5))
  model.add(Dense(5, kernel_initializer='normal', activation='tanh'))
  model.add(Dropout(0.5))
  model.add(Dense(3, kernel_initializer='normal', activation='tanh'))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=arch4, epochs=100, batch_size=10, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, df_x, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))